In this challenge you must analyze demographic data using Pandas. You are given a dataset of demographic data that was extracted from the 1994 Census database.

You must use Pandas to answer the following questions:
* How many people of each race are represented in this dataset? This should be a Pandas series with race names as the index labels. (`race` column)
* What is the average age of men?
* What is the percentage of people who have a Bachelor's degree?
* What percentage of people with advanced education (`Bachelors`, `Masters`, or `Doctorate`) make more than 50K?
* What percentage of people without advanced education make more than 50K?
* What is the minimum number of hours a person works per week?
* What percentage of the people who work the minimum number of hours per week have a salary of more than 50K?
* What country has the highest percentage of people that earn >50K and what is that percentage?
* Identify the most popular occupation for those who earn >50K in India. 

In [5]:
import pandas as pd


def calculate_demographic_data(print_data=True):
    # Read data from file
    df = pd.read_csv('adult.data.csv')

    # How many of each race are represented in this dataset? This should be a Pandas series with race names as the index labels.
    race_count = df['race'].value_counts()

    # What is the average age of men?
    average_age_men1 = df[df['sex']=='Male']['age'].mean()
    average_age_men=round(average_age_men1,1)
    # What is the percentage of people who have a Bachelor's degree?
    percentage_bachelors1 = (df.loc[df['education']=='Bachelors','education'].count())*100/(df['education'].count())
    percentage_bachelors=round(percentage_bachelors1,1)

    # What percentage of people with advanced education (`Bachelors`, `Masters`, or `Doctorate`) make more than 50K?
    # What percentage of people without advanced education make more than 50K?

    # with and without `Bachelors`, `Masters`, or `Doctorate`
    higher_education = df[df['education'].isin(['Bachelors','Masters','Doctorate'])]
    lower_education = df[~(df['education'].isin(['Bachelors','Masters','Doctorate']))]

    # percentage with salary >50K
    #higher_education_rich = higher_education.loc[higher_education['salary']=='>50K','education'].count()*100/(df['education'].count())
    higher_education_rich=round((higher_education.loc[higher_education['salary']=='>50K','age'].count()*100/higher_education.shape[0]),1)

    lower_education_rich = round((lower_education.loc[lower_education['salary']=='>50K','age'].count()*100/lower_education.shape[0]),1)

    # What is the minimum number of hours a person works per week (hours-per-week feature)?
    min_work_hours = df['hours-per-week'].min()

    # What percentage of the people who work the minimum number of hours per week have a salary of >50K?
    num_min_workers = len(df[df['hours-per-week']==min_work_hours])

    rich_percentage = round(len(df[(df['hours-per-week'] == min_work_hours) & (df['salary'] == '>50K')]) / num_min_workers * 100, 1)

    # What country has the highest percentage of people that earn >50K?
    aggregation = {
        '>50K':  ('salary', lambda x: (x == ">50K").sum()),
        '<=50K': ('salary', lambda x: (x == "<=50K").sum()),
                  }

# Creating a new table that has native-country as the index and columns that have the counts of >50K and <=50K
    df2 = df.groupby('native-country').agg(**aggregation)

# Function that gets the total per country
    def getTotal(row):
        return row['>50K'] + row['<=50K']

# Add a column that calculate the % of those that earn >50K
    df2['>50K%'] = df2.apply(lambda row: (row['>50K'] / getTotal(row) *100).round(1), axis=1)

# Sorting the values by >50K% - largest number at the top
    df2 = df2.sort_values(">50K%",ascending=False)

# Get the name of the country and put into Title Case
    highest_earning_country = df2.iloc[0].name.title()

# Get the percentage
    highest_earning_country_percentage = df2.iloc[0][">50K%"]

  


    # Identify the most popular occupation for those who earn >50K in India.
   
    top_IN_occupation = df.loc[((df['native-country']=='India') & (df['salary']=='>50K')),'occupation'].value_counts().idxmax()


    if print_data:
        print("Number of each race:\n", race_count) 
        print("Average age of men:", average_age_men)
        print(f"Percentage with Bachelors degrees: {percentage_bachelors}%")
        print(f"Percentage with higher education that earn >50K: {higher_education_rich}%")
        print(f"Percentage without higher education that earn >50K: {lower_education_rich}%")
        print(f"Min work time: {min_work_hours} hours/week")
        print(f"Percentage of rich among those who work fewest hours: {rich_percentage}%")
        print("Country with highest percentage of rich:", highest_earning_country)
        print(f"Highest percentage of rich people in country: {highest_earning_country_percentage}%")
        print("Top occupations in India:", top_IN_occupation)


In [6]:
calculate_demographic_data()

Number of each race:
 White                 27816
Black                  3124
Asian-Pac-Islander     1039
Amer-Indian-Eskimo      311
Other                   271
Name: race, dtype: int64
Average age of men: 39.4
Percentage with Bachelors degrees: 16.4%
Percentage with higher education that earn >50K: 46.5%
Percentage without higher education that earn >50K: 17.4%
Min work time: 1 hours/week
Percentage of rich among those who work fewest hours: 10.0%
Country with highest percentage of rich: Iran
Highest percentage of rich people in country: 41.9%
Top occupations in India: Prof-specialty
